Extract the following fields and store the data as restaurants.csv.
* Restaurant Id
* Restaurant Name
* Country
* City
* User Rating Votes
* User Aggregate Rating (in float)
* Cuisines


In [12]:
import pandas as pd

#load the restaurant data from JSON
restaurant_data = pd.read_json(r'C:\Users\valkr\Desktop\Data Engineer CC4\gds_assessment\input\restaurant_data.json')

#display the first few rows to understand the structure
restaurant_data.head()


,results_found,results_start,results_shown,restaurants
0,29287,1,20,"[{'restaurant': {'R': {'res_id': 18649486}, 'a..."
1,7625,1,20,"[{'restaurant': {'R': {'res_id': 18707652}, 'a..."
2,21776,1,20,"[{'restaurant': {'R': {'res_id': 18392725}, 'a..."
3,16762,1,20,"[{'restaurant': {'R': {'res_id': 58882}, 'apik..."
4,12026,1,20,"[{'restaurant': {'R': {'res_id': 18893197}, 'a..."


In [4]:
#load country code data
country_codes = pd.read_excel(r'C:\Users\valkr\Desktop\Data Engineer CC4\gds_assessment\input\Country-Code.xlsx')

#display the first few rows to understand the structure
country_codes.head()


,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil
3,37,Canada
4,94,Indonesia


In [5]:
def normalise_restaurants(restaurants, country_codes):
    #to store data
    normalised_data = []
    #iterate through "restuarants"
    for restaurant_list in restaurants['restaurants']:
        #for each restaurant
        for restaurant in restaurant_list:
            restaurant_info = restaurant['restaurant']
            restaurant_json_country_code = restaurant_info['location']['country_id']
            #match restaurant_json_country_code with Country-Code.xlsx
            country_name = country_codes[country_codes['Country Code'] == restaurant_json_country_code]['Country'].values[0] if restaurant_json_country_code in country_codes['Country Code'].values else 'Unknown'

            normalised_data.append({
                'Restaurant Id': restaurant_info['R']['res_id'],
                'Restaurant Name': restaurant_info['name'],
                'Country': country_name,
                'City': restaurant_info['location']['city'],
                'User Rating Votes': restaurant_info['user_rating']['votes'],
                'User Aggregate Rating': float(restaurant_info['user_rating']['aggregate_rating']),
                'Cuisines': restaurant_info['cuisines']
            })

    return pd.DataFrame(normalised_data)

In [6]:
normalised_restaurants = normalise_restaurants(restaurant_data, country_codes)
#check structure
normalised_restaurants.head()

,Restaurant Id,Restaurant Name,Country,City,User Rating Votes,User Aggregate Rating,Cuisines
0,18649486,The Drunken Botanist,India,Gurgaon,4765,4.4,"Continental, Italian, North Indian, Chinese"
1,308322,Hauz Khas Social,India,New Delhi,13627,4.6,"Continental, American, Asian, North Indian, Ch..."
2,18856789,AIR- An Ivory Region,India,New Delhi,1819,4.1,"North Indian, Chinese, Continental, Asian"
3,307374,AMA Cafe,India,New Delhi,3252,4.4,"Cafe, Juices"
4,18238278,Tamasha,India,New Delhi,8112,4.4,"Finger Food, North Indian, Continental, Italian"


In [7]:
# Save data to CSV
normalised_restaurants.to_csv(r'C:\Users\valkr\Desktop\Data Engineer CC4\gds_assessment\output\restaurants.csv', index=False)

2. 	Extract the list of restaurants that have past event in the month of April 2019 and store the data as restaurant_events.csv.
* Event Id
* Restaurant Id
* Restaurant Name
* Photo URL
* Event Title
* Event Start Date
* Event End Date


In [23]:
from datetime import datetime
def normalise_events(restaurants):
    #store data
    events_data = []
    #make april in datetime
    start_april_2019 = datetime(2019, 4, 1)
    end_april_2019 = datetime(2019, 4, 30)
    #same iteration as above
    for restaurant_list in restaurants['restaurants']:
        for restaurant in restaurant_list:
            restaurant_info = restaurant['restaurant']
            if 'zomato_events' in restaurant_info:#check if there was an event
                for event in restaurant_info['zomato_events']:
                    event_info = event['event']
                    #format dates
                    start_date = pd.to_datetime(event_info['start_date'])
                    end_date = pd.to_datetime(event_info['end_date'])
                    #events that occurred in April 2019
                    #this mean so long as there was an event during april 2019, it counts
                    if start_date <= start_april_2019 and end_date >= end_april_2019:
                        events_data.append({
                            'Event Id': event_info['event_id'],
                            'Restaurant Id': restaurant_info['R']['res_id'],
                            'Restaurant Name': restaurant_info['name'],
                            'Photo URL': event_info['photos'][0]['photo']['url'] if event_info['photos'] else 'NA',
                            'Event Title': event_info['title'],
                            'Event Start Date': event_info['start_date'],
                            'Event End Date': event_info['end_date'],
                        })

    return pd.DataFrame(events_data)




In [26]:
events_normalised = normalise_events(restaurant_data)
events_normalised.head()
print(events_normalised)

    Event Id  Restaurant Id                 Restaurant Name  \
0     322331       18649486            The Drunken Botanist   
1     332812         308322                Hauz Khas Social   
2     116096       18466951                 Jungle Jamboree   
3     161417       18466951                 Jungle Jamboree   
4     332812       18246991                    Odeon Social   
..       ...            ...                             ...   
81    252193       18506282                       Sibyullee   
82    322902        7003663            Federal Delicatessen   
83    305850        7001670  Eight - Cordis, Auckland Hotel   
84    329300       18981544                        Katsu YA   
85    329301       18981544                        Katsu YA   

                                            Photo URL  \
0   https://b.zmtcdn.com/data/zomato_events/photos...   
1   https://b.zmtcdn.com/data/zomato_events/photos...   
2   https://b.zmtcdn.com/data/zomato_events/photos...   
3   https://b.z

In [25]:
# Save the events data to CSV
events_normalised.to_csv(r'C:\Users\valkr\Desktop\Data Engineer CC4\gds_assessment\output\restaurant_events.csv', index=False)